In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [5]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables
using GLM

In [3]:
includet("../VARs/utils.jl")

In [12]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

freq = 4

Δd = percentchange(data_source[:Dividend], :log)
dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])
cpi = percentchange(data_source[:CPI], :log) 

t_bill = data_source[:T_Bill]


data= merge(Δd, dy)

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq)) 
display(cor(values(s_data)))



,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Dividend,0.0314,0.1948,0.0551,0.0924,-0.2683,-1.4492,0.0257,0.8456,5.4914,1.7511,0.0591
Dividend_Stock_Price,0.8099,-11.1063,-14.4706,-13.2978,-17.9779,-0.148,-15.8163,0.9714,-0.8685,-17.8665,-14.0743


nothing

2×2 Matrix{Float64}:
 1.0        0.0171604
 0.0171604  1.0

In [17]:
s_data

279×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1955-03-01 to 2024-07-01
┌────────────┬─────────────┬──────────────────────┐
│            │ Dividend    │ Dividend_Stock_Price │
├────────────┼─────────────┼──────────────────────┤
│ 1955-03-01 │   0.0107139 │             -3.15268 │
│ 1955-06-01 │   0.0148193 │             -3.25267 │
│ 1955-09-01 │   0.0248975 │             -3.29013 │
│ 1955-12-01 │   0.0262886 │             -3.30445 │
│ 1956-03-01 │   0.0486932 │             -3.31963 │
│ 1956-06-01 │   0.0336481 │             -3.25434 │
│ 1956-09-01 │ -0.00367956 │             -3.22292 │
│ 1956-12-01 │  -0.0395159 │             -3.29113 │
│     ⋮      │      ⋮      │          ⋮           │
│ 2023-03-01 │   0.0151287 │             -4.09598 │
│ 2023-06-01 │  0.00778484 │             -4.16793 │
│ 2023-09-01 │   0.0105703 │              -4.1202 │
│ 2023-12-01 │   0.0119053 │             -4.21477 │
│ 2024-03-01 │   0.0103198 │              -4.3012 │
│ 2024-06-01 │    0.010718 │            

In [16]:
TimeSeries.lag(s_data[:Dividend])

278×1 TimeArray{Float64, 1, Date, Vector{Float64}} 1955-06-01 to 2024-07-01
┌────────────┬─────────────┐
│            │ Dividend    │
├────────────┼─────────────┤
│ 1955-06-01 │   0.0107139 │
│ 1955-09-01 │   0.0148193 │
│ 1955-12-01 │   0.0248975 │
│ 1956-03-01 │   0.0262886 │
│ 1956-06-01 │   0.0486932 │
│ 1956-09-01 │   0.0336481 │
│ 1956-12-01 │ -0.00367956 │
│ 1957-03-01 │  -0.0395159 │
│     ⋮      │      ⋮      │
│ 2023-03-01 │     0.02249 │
│ 2023-06-01 │   0.0151287 │
│ 2023-09-01 │  0.00778484 │
│ 2023-12-01 │   0.0105703 │
│ 2024-03-01 │   0.0119053 │
│ 2024-06-01 │   0.0103198 │
│ 2024-07-01 │    0.010718 │
└────────────┴─────────────┘
            263 rows omitted